In [1]:
import os
import re
import sys
import tensorflow as tf
import tensorflow.python.platform
from tensorflow.python.platform import gfile
import numpy as np
import pandas as pd
import sklearn
from sklearn import cross_validation
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC, LinearSVC
import matplotlib.pyplot as plt
import pickle
import av

In [2]:
path_to_video = 'test.mp4'
path_to_frames = 'frames/'
images_dir = path_to_frames
model_dir = '../imagenet/classify_image_graph_def.pb'

In [3]:
def saliency_vector(features):
    features = preprocessing.normalize(features, norm='l2')
    feature_1 = np.vstack((np.zeros((1,features.shape[1])),features))
    feature_2 = np.vstack((features,np.zeros((1,features.shape[1]))))
    diff_vector = feature_2 - feature_1
    return diff_vector

In [4]:
def frame_extractor(path_to_video,path_to_frames):
    container = av.open(path_to_video)
    video = next(s for s in container.streams if s.type == b'video')
    for packet in container.demux(video):
        for frame in packet.decode():
            frame.to_image().save(path_to_frames+'/frame-%06d.jpg' % frame.index)

In [5]:
class ExtractFeatures:
    def __init__(self,model_path,images_dir):
        self.model = model_path
        self.list_images = [images_dir+f for f in os.listdir(images_dir) if re.search('jpg|JPG', f)]
        self.list_images.sort()
    
    def create_graph(self):
        with gfile.FastGFile(self.model, 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
            _ = tf.import_graph_def(graph_def, name='')
    
    def get_layer_list(self):
        self.create_graph()
        with tf.Session() as sess:
            operation_list = sess.graph.get_operations()
            for op in operation_list:
                print op.name
                print op.values().get_shape()

    def extract_features(self,layer_name='pool_3:0',nb_features=2048):
        features = np.empty((len(self.list_images),nb_features))
        labels = []
        self.create_graph()
        with tf.Session() as sess:
            next_to_last_tensor = sess.graph.get_tensor_by_name(layer_name)
            for ind, image in enumerate(self.list_images):
                #if (ind%100 == 0):
                print('Processing %s...' % (image))
                if not gfile.Exists(image):
                    tf.logging.fatal('File does not exist %s', image)
                    continue
                image_data = gfile.FastGFile(image, 'rb').read()
                predictions = sess.run(next_to_last_tensor,{'DecodeJpeg/contents:0': image_data})
                features[ind,:] = np.squeeze(predictions)
        return features

In [6]:
if __name__ == "__main__":
    #frame_extractor(path_to_video,path_to_frames)
    extract_features = ExtractFeatures(model_dir,images_dir)
    extract_features.create_graph()
    features = extract_features.extract_features()
    pickle.dump(features, open('features', 'wb'))
    diff_vector = saliency_vector(features)
    pickle.dump(diff_vector, open('diff_vector','wb'))

Processing frames/frame-0000.jpg...
Processing frames/frame-0001.jpg...
Processing frames/frame-0002.jpg...
Processing frames/frame-0169.jpg...
Processing frames/frame-4091.jpg...
Processing frames/frame-6695.jpg...


In [7]:
print diff_vector.shape

(7, 2048)


In [8]:
distance = []
for vector in diff_vector:
    distance.append(np.linalg.norm(vector))
print distance

[1.0000000000000004, 0.1230300400139373, 0.036877493381364618, 0.99887036188231781, 0.93332663686498085, 0.48760886276019016, 0.99999999999999944]


In [9]:
print extract_features.list_images

['frames/frame-0000.jpg', 'frames/frame-0001.jpg', 'frames/frame-0002.jpg', 'frames/frame-0169.jpg', 'frames/frame-4091.jpg', 'frames/frame-6695.jpg']
